# EO4SD SHORELINE CHANGE MAPPING AND FORECASTING

This code has been modifed by Carpenter (2020) for the project Earth Observation for Sustainable Development. Below demonstrates an example processing workflow for Benin and Togo's Coastline between 2000-2020.

This software is based on scripts and code developed by:
* Vos K., Splinter K.D., Harley M.D., Simmons J.A., Turner I.L. (2019). CoastSat: a Google Earth Engine-enabled Python toolkit to extract shorelines from publicly available satellite imagery. Environmental Modelling and Software. 122, 104528. https://doi.org/10.1016/j.envsoft.2019.104528

It enables the users to extract time-series of shoreline change over the last 20+ years at their site of interest.
There are three main steps:
1. Retrieval of median composite satellite images of the region of interest from Google Earth Engine
2. Shoreline extraction at sub-pixel resolution

## Initial settings

Refer to the Set-up and Installation section of the User Handbook for instructions on how to install the Python packages necessary to run the software, including Google Earth Engine Python API. See original methodology via https://github.com/kvos/CoastSat

In [ ]:
import os
import numpy as np
import pickle
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from coastsat import NOC_tools, NOC_shoreline, NOC_download, NOC_preprocess, \
    SDS_tools, SDS_shoreline, SDS_download, SDS_preprocess

settings ={'output_epsg': 32620,
           
           'cloud_thresh':0.9,
           
            # [ONLY FOR ADVANCED USERS] shoreline detection parameters:
            'min_beach_area': 4500,     # minimum area (in metres^2) for an object to be labelled as a beach
            'buffer_size': 150,         # radius (in metres) for buffer around sandy pixels considered in the shoreline detection
            'min_length_sl': 200,       # minimum length (in metres) of shoreline perimeter to be valid
            'cloud_mask_issue': False,  # switch this parameter to True if sand pixels are masked (in black) on many images  
    
            # Sentinel
            'CLOUD_FILTER': 50,         # [Integer] Maximum image cloud cover percent allowed in image collection'
            'CLD_PRB_THRESH': 25,       # {Integer] Cloud probability (%); values greater than are considered cloud
            'NIR_DRK_THRESH': 0.08,     # [Float] Near-infrared reflectance; values less than are considered potential cloud shadow
            'CLD_PRJ_DIST': 2,          # [Float] Maximum distance (km) to search for cloud shadows from cloud edges |
            'BUFFER': 50,               # [Integer] Distance (m) to dilate the edge of cloud-identified objects |

            'reference_shoreline': True,
            'max_dist_ref': 200,        # max distance (in meters) allowed from the reference shoreline
   
           # labels for classes for the classifier and colours for display
           'classes':{'hard': [1, [1, 0, 0]], 'nature': [2, [0, 1, 0]],
                      'urban': [3, [1, 1, 0]], 'white-water': [4, [1, 0, 1]],
                      'water': [5, [0, 0.4, 1]], 'sand': [6, [1, 0.6, 0]]},                                             
          }

## 1. Retrieval of the images from GEE

To retrieve from the GEE server the available satellite images cropped around the user-defined region of coastline for the particular time period of interest, the following variables are required:

* Coordinate list: a list of the coordinates of the region of interest (longitude/latitude pairs in WGS84) – see section 'create coordinate list in User handbook to extract ROI from study site
* all_dates: dates over which the images will be retrieved (e.g., dates = ['2017-12-01', '2018-01-01'])
* all_sats: satellite missions to consider (e.g., sat_list = ['L7', 'L8', 'S2'] for Landsat 7, 8 and Sentinel-2 collections)
* sitename: name of the site (this is the name of the subfolder where the images and other accompanying files will be stored)

Similar to coastsat, there are some extra parameters such as those to go through manual checking, whether to combine Landsat collections and cloud thresholds.

Make sure the area of your ROI is smaller than 100 km2 (if larger split it into smaller ROIs) - GEE limits download size

In [ ]:
# directory where the data will be accessed and stored
dataPartition = 'c:\\data\\coastsat'
country = 'UK'
base_filepath = os.path.join(dataPartition, country)  

# filepaths etc
filepath_sites = os.path.join(base_filepath, 'sites')
sites = os.listdir(filepath_sites)

sat_name = 'S2'

dates = ['2020-01-01', '2020-06-30']

for site in sites:

    kml_filepath = os.path.join(filepath_sites, site)
    kml_polygon = NOC_tools.polygon_from_kml(kml_filepath)
    roi_polygon = SDS_tools.smallest_rectangle(kml_polygon)
    
    sitename = site[:site.find('.')]

    inputs = {'polygon': roi_polygon, 'dates': dates, 'sat_list': [sat_name],
              'sitename': sitename, 'filepath': base_filepath, 'threshold': 0}
    
    settings['inputs'] = inputs
    
    print(f'Processing: {sitename}')

In [ ]:
    ## download median images
    #metadata = NOC_download.retrieve_median_optical(settings)
    
    print('finished ...')

In [ ]:
    ## load saved metadata
    metadata = NOC_download.get_metadata(inputs)
    
    print('finished ...')

In [ ]:
SDS_preprocess.save_jpg(metadata, settings)

In [ ]:
settings['reference_shoreline'] = NOC_preprocess.get_reference_shoreline(inputs)

In [ ]:
    %matplotlib qt
    
    ##Batch shoreline detection
    output = NOC_shoreline.extract_shorelines_optical(metadata, settings)
    
    print('finished ...')

In [ ]:
output = SDS_tools.remove_duplicates(output) # removes duplicates (images taken on the same date by the same satellite)
output = SDS_tools.remove_inaccurate_georef(output, 10) # remove inaccurate georeferencing (set threshold to 10 m)

In [ ]:
gdf = SDS_tools.output_to_gdf(output, 'lines')

satname = output['satname'][0]
file_string = f'{sitename}_shorelines_{satname}.geojson'

gdf.crs = {'init':'epsg:'+str(settings['output_epsg'])} # set layer projection
# save GEOJSON layer to file
gdf.to_file(os.path.join(inputs['filepath'], inputs['sitename'],
                         file_string),
                         driver='GeoJSON', encoding='utf-8')
print('finished ...')

In [ ]:
fig = plt.figure(figsize=[15,8])
plt.axis('equal')
plt.xlabel('Eastings')
plt.ylabel('Northings')
plt.grid(linestyle=':', color='0.5')
for i in range(len(output['shorelines'])):
    sl = output['shorelines'][i]
    date = output['dates'][i]
    plt.plot(sl[:,0], sl[:,1], '.', label=date.strftime('%d-%m-%Y'))
plt.legend();